In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import numpy as np
import labrotation.file_handling as fh
import os
import pandas as pd
import labrotation.two_photon_session as tps
import datadoc_util
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime as dt

In [ ]:
save_figs = True  # set to True to save the figures created
save_as_eps = False
if save_as_eps:
    file_format = ".eps"
else:
    file_format = ".jpg"
if save_figs:
    print(f"Going to save figures as {file_format} files.")

In [ ]:
def get_datetime_for_fname():
    now = dt.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
df_colors = ddoc.getColorings()

In [ ]:
dict_colors_mouse = df_colors[["mouse_id", "color"]].to_dict(orient="list")
dict_colors_mouse = dict(zip(dict_colors_mouse["mouse_id"], dict_colors_mouse["color"]))

In [ ]:
ddoc.getSegmentsForUUID("04b8cfbfa1c347058bb139b4661edcf1")

In [ ]:
used_mouse_ids = ["OPI-2239", "WEZ-8917", "WEZ-8924", "WEZ-8922"]

In [ ]:
event_traces_fpath = fh.open_file("Open .h5 file containing assembled traces for all seizures!")
print(event_traces_fpath)

In [ ]:
traces_dict = dict()  
traces_meta_dict = dict()
# first keys are event uuids, inside the following dataset names:
# 'lfp_mov_t', 'lfp_mov_y', 'lfp_t', 'lfp_y', 'lv_dist', 'lv_rounds', 
# 'lv_running', 'lv_speed', 'lv_t_s', 'lv_totdist', 'mean_fluo'
with h5py.File(event_traces_fpath, "r") as hf:
    for uuid in hf.keys():
        if hf[uuid].attrs["mouse_id"] in used_mouse_ids:
            session_dataset_dict = dict() 
            session_meta_dict = dict()
            for dataset_name in hf[uuid].keys():
                session_dataset_dict[dataset_name] = np.array(hf[uuid][dataset_name])
            for attr_name in hf[uuid].attrs:
                session_meta_dict[attr_name] = hf[uuid].attrs[attr_name]
            traces_dict[uuid] = session_dataset_dict.copy()
            traces_meta_dict[uuid] = session_meta_dict.copy()

In [ ]:
exptype_wintype_id_dict = {}   # keys: experiment_type, window_type, mouse_id, value: [uuid1, uuid2, ...]
for uuid in traces_meta_dict.keys():
    exp_type = traces_meta_dict[uuid]["exp_type"]
    win_type = traces_meta_dict[uuid]["window_type"]
    mouse_id = traces_meta_dict[uuid]["mouse_id"]
    if exp_type not in exptype_wintype_id_dict.keys():
        exptype_wintype_id_dict[exp_type] = dict()
    if win_type not in exptype_wintype_id_dict[exp_type].keys():
        exptype_wintype_id_dict[exp_type][win_type] = dict()
    if mouse_id not in exptype_wintype_id_dict[exp_type][win_type].keys():
        exptype_wintype_id_dict[exp_type][win_type][mouse_id] = []  # list of uuids
    exptype_wintype_id_dict[exp_type][win_type][mouse_id].append(uuid)

In [ ]:
exptype_wintype_id_dict.keys()

In [ ]:
df_colors

In [ ]:
AMPLITUDE = 100.0
offset = 0
fig = plt.figure(figsize=(18,18))
mouse_ids = traces_dict.keys()
prev_range = 0.0

for exp_type in exptype_wintype_id_dict.keys():
    for win_type in exptype_wintype_id_dict[exp_type].keys():
        for mouse_id in exptype_wintype_id_dict[exp_type][win_type].keys():
            for event_uuid in exptype_wintype_id_dict[exp_type][win_type][mouse_id]:
                df_segments = ddoc.getSegmentsForUUID(event_uuid)
                i_frame_stim_begin = df_segments[df_segments["interval_type"] == "stimulation"].frame_begin.iloc[0] - 1
                i_frame_stim_end = df_segments[df_segments["interval_type"] == "stimulation"].frame_end.iloc[0] - 1  # in 1 indexing, inclusive
                
                nik_trace = traces_dict[event_uuid]["mean_fluo"]
                
                nik_trace[i_frame_stim_begin:i_frame_stim_end] = 1.2*max(nik_trace[i_frame_stim_end+1:])  # reduce stim amplitude
                
                min_nik = min(nik_trace)
                max_nik = max(nik_trace)
                
                t = traces_dict[event_uuid]["lv_t_s"]
                
                color = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
                plt.plot(t, AMPLITUDE*(nik_trace- min_nik)/(max_nik - min_nik)+offset, color=color)
                offset+=1.1*AMPLITUDE
                if (len(traces_dict[event_uuid]["mean_fluo"]) != 13483):
                    print(f'{event_uuid}: {len(traces_dict[event_uuid]["mean_fluo"])}, {mouse_id}, {exp_type}')





#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
#plt.xlim((300, 460))  # 250, 500
if save_figs:
    out_fpath = f"D:\\Downloads\\traces_normalized_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
    print(f"Saved as {out_fpath}")
plt.show()

### Plot Nik + Loco

In [ ]:
def plotNikLoco(exp_type, show_segments=True, bl_equal_post = True, n_bl_frames=4500):
    AMPLITUDE = 100.0
    offset = 0
    
    n_recordings_with_type =  0
    for win_type in exptype_wintype_id_dict[exp_type].keys():
        for mouse_id in exptype_wintype_id_dict[exp_type][win_type].keys():
            for event_uuid in exptype_wintype_id_dict[exp_type][win_type][mouse_id]:
                n_recordings_with_type += 1
    fig = plt.figure(figsize=(18,n_recordings_with_type*3))
    mouse_ids = traces_dict.keys()
    prev_range = 0.0
    for win_type in exptype_wintype_id_dict[exp_type].keys():
        for mouse_id in exptype_wintype_id_dict[exp_type][win_type].keys(): 
            for event_uuid in exptype_wintype_id_dict[exp_type][win_type][mouse_id]:
                df_segments = ddoc.getSegmentsForUUID(event_uuid)
                i_frame_stim_begin = df_segments[df_segments["interval_type"] == "stimulation"].frame_begin.iloc[0] - 1
                i_frame_stim_end = df_segments[df_segments["interval_type"] == "stimulation"].frame_end.iloc[0] - 1  # in 1 indexing, inclusive
                
                
                
                
                nik_trace = traces_dict[event_uuid]["mean_fluo"]

                nik_trace[i_frame_stim_begin:i_frame_stim_end] = 1.2*max(nik_trace[i_frame_stim_end+1:])  # reduce stim amplitude

                min_nik = min(nik_trace)
                max_nik = max(nik_trace)

                t = traces_dict[event_uuid]["lv_t_s"]
                
                if show_segments:
                    # get begin and end time points of baseline and post-stim segments 
                    i_frame_bl_end = i_frame_stim_begin
                    i_frame_post_begin = i_frame_stim_end
                    if n_bl_frames < i_frame_stim_begin:
                        i_frame_bl_begin = i_frame_bl_end - n_bl_frames 
                    else:
                        i_frame_bl_begin = 0
                    if bl_equal_post:
                        i_frame_post_end = i_frame_post_begin + n_bl_frames
                    if not bl_equal_post or i_frame_post_end > len(nik_trace):
                        i_frame_post_end = len(nik_trace) - 1
                    # plot them
                    plt.vlines(x=[t[i_frame_bl_begin], t[i_frame_bl_end], t[i_frame_post_begin], t[i_frame_post_end]], ymin = offset, ymax = offset+2.2*AMPLITUDE)

                labview_trace = traces_dict[event_uuid]["lv_speed"]
                min_lv = min(labview_trace)
                max_lv = max(labview_trace)
                plt.plot(t, AMPLITUDE*(labview_trace - min_lv)/(max_lv - min_lv)+offset, color="black")
                offset +=1.1*AMPLITUDE
                
                

                color = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
                plt.plot(t, AMPLITUDE*(nik_trace- min_nik)/(max_nik - min_nik)+offset, color=color)
                offset+=1.1*AMPLITUDE

                



                if (len(traces_dict[event_uuid]["mean_fluo"]) != 13483):
                    print(f'{event_uuid}: {len(traces_dict[event_uuid]["mean_fluo"])}, {mouse_id}, {exp_type}')




    print(f"Total: {n_recordings_with_type} traces")
    plt.suptitle(exp_type, fontsize=22)
    #plt.axis("off")
    plt.yticks([])
    plt.xlabel("Time (s)", fontsize=14)
    plt.tight_layout()
    #plt.xlim((300, 460))  # 250, 500
    if save_figs:
        out_fpath = f"D:\\Downloads\\traces_normalized_{exp_type}_{get_datetime_for_fname()}{file_format}"
        plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
        print(f"Saved as {out_fpath}")
    plt.show()
    

In [ ]:
def compareLoco(exp_type):
    AMPLITUDE = 100.0
    offset = 0
    n_recordings_with_type =  0
    for win_type in exptype_wintype_id_dict[exp_type].keys():
        for mouse_id in exptype_wintype_id_dict[exp_type][win_type].keys():
            for event_uuid in exptype_wintype_id_dict[exp_type][win_type][mouse_id]:
                n_recordings_with_type += 1
    fig = plt.figure(figsize=(18,n_recordings_with_type*3))
    mouse_ids = traces_dict.keys()
    prev_range = 0.0
    for win_type in exptype_wintype_id_dict[exp_type].keys():
        for mouse_id in exptype_wintype_id_dict[exp_type][win_type].keys():
            for event_uuid in exptype_wintype_id_dict[exp_type][win_type][mouse_id]:
                df_segments = ddoc.getSegmentsForUUID(event_uuid)
                i_frame_stim_begin = df_segments[df_segments["interval_type"] == "stimulation"].frame_begin.iloc[0] - 1
                i_frame_stim_end = df_segments[df_segments["interval_type"] == "stimulation"].frame_end.iloc[0] - 1  # in 1 indexing, inclusive

                lfp_mov_trace = traces_dict[event_uuid]["lfp_mov_y"]
                t_lfp = traces_dict[event_uuid]["lfp_mov_t"]
                lfp_min = min(lfp_mov_trace)
                lfp_max = max(lfp_mov_trace)
                

                t = traces_dict[event_uuid]["lv_t_s"]



                labview_trace = traces_dict[event_uuid]["lv_speed"]
                min_lv = min(labview_trace[10:])
                max_lv = max(labview_trace[10:])
                plt.plot(t_lfp, AMPLITUDE*(lfp_mov_trace - lfp_min)/(lfp_max - lfp_min)+offset, color="black")
                offset +=1.1*AMPLITUDE

                if t[0] +1000 < t_lfp[0]:
                    print(f"{event_uuid} {mouse_id}")
                    
                
                color = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
                plt.plot(t, AMPLITUDE*(labview_trace- min_lv)/(max_lv - min_lv)+offset, color=color)
                offset+=1.1*AMPLITUDE





                if (len(traces_dict[event_uuid]["mean_fluo"]) != 13483):
                    print(f'{event_uuid}: {len(traces_dict[event_uuid]["mean_fluo"])}, {mouse_id}, {exp_type}')





    #plt.axis("off")
    plt.suptitle(exp_type, fontsize=22)
    plt.yticks([])
    plt.xlabel("Time (s)", fontsize=14)
    #plt.xlim((300, 460))  # 250, 500
    plt.tight_layout()
    if save_figs:
        out_fpath = f"D:\\Downloads\\traces_normalized_{get_datetime_for_fname()}{file_format}"
        plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
        print(f"Saved as {out_fpath}")
    plt.show()

In [ ]:
exptype_wintype_id_dict.keys()

In [ ]:
plotNikLoco("chr2_ctl")

## Compare LFP and LabView loco to check time matching

In [ ]:
compareLoco("chr2_sd")